# 7 Expressions 

The `tables.Expr` class evaluates (in-kernel) expressions on array-like objects. All the internal computations are performed via the `Numexpr` package. `Numexpr` provides multi-threading, SIMD and blocking techniques to solve the starving CPU problem. In combination with compressors (blosc) very high out-of-core performance can be reached for expressions on large-than-memory arrays (tables).

In [1]:
import tables
import numpy as np

In [2]:
data_dir = 'expr'
import os
import shutil
if os.path.exists(data_dir):
    shutil.rmtree(data_dir)
os.mkdir(data_dir)

We create a table with four columns (four-momentum from particle physics) and store random floats:

In [3]:
FILENAME = os.path.join(data_dir, "momentum.h5")
f = tables.open_file(FILENAME, "w")

In [4]:
class FourMomentum(tables.IsDescription): 
    E = tables.Float64Col()
    p_x = tables.Float64Col()
    p_y = tables.Float64Col()
    p_z = tables.Float64Col() 

In [5]:
filters = tables.Filters(complevel=0)  # no compression

In [6]:
t = f.create_table(f.root, "mydata", FourMomentum, filters=filters)

In [7]:
dtype = t.dtype

Store 1 million rows:

In [8]:
N = int(1e6)

In [9]:
arr = np.random.random((N,)).astype(dtype)
arr[:2]

array([( 0.65621637,  0.65621637,  0.65621637,  0.65621637),
       ( 0.33005409,  0.33005409,  0.33005409,  0.33005409)],
      dtype=[('E', '<f8'), ('p_x', '<f8'), ('p_y', '<f8'), ('p_z', '<f8')])

In [10]:
t.append(arr)

In [11]:
t.flush()

In [12]:
t[:10]['p_x']

array([ 0.65621637,  0.33005409,  0.59829078,  0.33769868,  0.77929896,
        0.35366154,  0.04231687,  0.85648859,  0.42816492,  0.22218853])

We can acccess the columns using the `Cols` accessor:

In [13]:
px = t.cols.p_x
py = t.cols.p_y
pz = t.cols.p_z

Define the expression:

In [14]:
expr = tables.Expr('px**2 + py**2 + pz**2')
expr

Evaluate the expression, result will be stored in-memory:

In [15]:
%timeit expr.eval()

89.8 ms ± 635 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Let's compare the pure-numpy version of the expression:

In [16]:
%timeit arr['p_x']**2 + arr['p_y']**2 + arr['p_z']**2

32.3 ms ± 3.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In this example numpy is 2-3x times faster. For large, in-memory arrays, pure numpy is usually about 1.5x faster.

### out-of-core

We can store results on-disk (in HDF5) so we can evaluate expressions out-of-core even if the results do not fit into memory:

In [17]:
output_array = f.create_carray(f.root, "output", atom=tables.Float64Atom(), shape=(N,), filters=filters)
output_array

/output (CArray(1000000,)) ''
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := (8192,)

In [18]:
expr.set_output(output_array)

In [19]:
%timeit expr.eval()

92.1 ms ± 773 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Read the output:

In [20]:
f.root.output[:]

array([ 1.29185977,  0.3268071 ,  1.07385558, ...,  0.87298304,
        2.27123376,  0.73753768])

In this way we can evaluate expressions with larger-than-memory results out-of-core. 

In [21]:
f.close()

## Using compression

The facilitate experimenting with dataset size, compression etc, we define some functions:

In [22]:
fn = os.path.join(data_dir, 'momentum.h5')

In [23]:
def create_dataset(N, filename, filters):
    """Create table '/mydata' with a random four-momentum table of size N"""
    with tables.open_file(filename, "w") as f:

        t = f.create_table(f.root, "mydata", FourMomentum, filters=filters)

        dtype = t.dtype
        arr = np.random.random((N,)).astype(dtype)
        t.append(arr)
        t.flush()

        f.create_carray(f.root, "output", atom=tables.Float64Atom(), shape=(N,), filters=filters)
        

In [24]:
def create_expression(f, output=None, expr='px**2 + py**2 + pz**2'):
    """Create an expression object"""
    t = f.root.mydata
    px = t.cols.p_x
    py = t.cols.p_y
    pz = t.cols.p_z

    e = tables.Expr(expr)
    if output is not None:
        e.set_output(output)
    return e


In [40]:
filters = tables.Filters(complevel=0)
fn = os.path.join(data_dir, 'momentum-uncompressed.h5')

In [41]:
N=int(1e7)  # N == 1e7 is about 300 Mbytes.
create_dataset(N, fn, filters)

In [42]:
!ls -lh {data_dir}

total 484M
-rw-r--r-- 1 tomkooij 197613 140M Jun 26 12:02 momentum-blosc-lz4.h5
-rw-r--r-- 1 tomkooij 197613 306M Jun 26 12:02 momentum-uncompressed.h5
-rw-r--r-- 1 tomkooij 197613  39M Jun 26 11:58 momentum.h5


In [43]:
with tables.open_file(fn, 'a') as f:
    expr = create_expression(f)
    %timeit expr.eval()

900 ms ± 59.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
with tables.open_file(fn, 'a') as f:
    expr = create_expression(f, output=f.root.output)
    %timeit expr.eval()

988 ms ± 79.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
!ls -lh {data_dir}

total 560M
-rw-r--r-- 1 tomkooij 197613 140M Jun 26 12:02 momentum-blosc-lz4.h5
-rw-r--r-- 1 tomkooij 197613 382M Jun 26 12:02 momentum-uncompressed.h5
-rw-r--r-- 1 tomkooij 197613  39M Jun 26 11:58 momentum.h5


# Exercise

Create an (reasonably) compressible dataset and investigate the `tables.Expr` performance with and without compression. 

Can you achieve reasonable perfomance?

In [38]:
#
#
# Solution starts here
#
#

In [34]:
filters = tables.Filters(complevel=6, complib='blosc:lz4')
fn = os.path.join(data_dir, 'momentum-blosc-lz4.h5')

In [35]:
N=int(1e7)  
create_dataset(N, fn, filters)

In [36]:
!ls -lh {data_dir}

total 413M
-rw-r--r-- 1 tomkooij 197613  69M Jun 26 12:01 momentum-blosc-lz4.h5
-rw-r--r-- 1 tomkooij 197613 306M Jun 26 12:01 momentum-uncompressed.h5
-rw-r--r-- 1 tomkooij 197613  39M Jun 26 11:58 momentum.h5


In [39]:
with tables.open_file(fn, 'a') as f:
    expr = create_expression(f, output=f.root.output)
    %timeit expr.eval()

1.32 s ± 50.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In general in HDF5 a compressed dataset performs slightly slower than the uncompressed dataset. This is somewhat disappointing as there are very efficient datacontairs (`bcolz`) which do achieve better compressed read/write performance.

However, in general, we can use compressors to give us very good compression, with only a slight read/write penalty.
